# Summarize titers across all sera and groups
Make summary plots for the titers from all sera.

In [ ]:
import altair as alt

import neutcurve
import numpy as np
import pandas as pd
from scipy import stats

_ = alt.data_transformers.disable_max_rows()

Get variables from `snakemake`

In [ ]:
#These are our input files. May want to add a file with the conditions so that we can plot by condition
input_titers = snakemake.input.input_titers
viral_strain_plot_order = snakemake.input.viral_strain_plot_order
sample_metadata_file = snakemake.input.sample_metadata_file
HAI_titers = snakemake.input.HAI_titers_file

#These are the output files which are the data in html
titers_chart_html = snakemake.output.titers_chart_html
titers_chart_1xVax_html = snakemake.output.titers_chart_1xVax_html
titers_chart_2xVax_html = snakemake.output.titers_chart_2xVax_html
titers_chart_selectedpeople_html = snakemake.output.titers_chart_selectedpeople_html
titers_chart_median_w182_html = snakemake.output.titers_chart_median_w182_html
titers_chart_median_all_html = snakemake.output.titers_chart_median_all_html
titers_chart_foldchange_html = snakemake.output.titers_chart_foldchange_html
titers_chart_splitbygroup_w182_html = snakemake.output.titers_chart_splitbygroup_w182_html
titers_chart_splitbygroup_all_html = snakemake.output.titers_chart_splitbygroup_all_html
titers_chart_selectedserum_rotated_html = snakemake.output.titers_chart_selectedserum_rotated_html
titers_chart_serawithgap_rotated_html = snakemake.output.titers_chart_serawithgap_rotated_html

In [ ]:
#Read in titer csv as a dataframe and get the order to plot the viruses, add metadata to sample file
titers = pd.read_csv(input_titers)
HAI_titers = pd.read_csv(HAI_titers)
sample_metadata = pd.read_csv(sample_metadata_file)
viruses_to_plot = pd.read_csv(viral_strain_plot_order)
assert len(titers) == len(titers.groupby(["serum", "virus"]))
viruses = viruses_to_plot.strain.tolist()
titers_withgroups = titers.merge(sample_metadata, on= "serum")
titers_withgroups['participant'] = "Participant " + titers_withgroups['individual']

In [ ]:
titers_withgroups['timepoint'] = titers_withgroups['serum'].str[7:].astype(int)
titers_withgroups['pID'] = titers_withgroups['serum'].str[:6]
titers_withgroups_vaccineonly = titers_withgroups.loc[titers_withgroups['virus'].str.contains("A/Hawaii/70/2019")]
titers_withgroups_vaccine = titers_withgroups_vaccineonly.rename(columns={"titer":"NGS_titer"})
HAI_titers = HAI_titers.rename(columns={"titer":"HAI_titer","strain":"virus"})

In [ ]:
#Make a dataframe with both HAI and NGS neutralization assay titers, look at Pearson correlation
compare_HAI = titers_withgroups_vaccine.merge(HAI_titers, on=["pID","timepoint","virus"])
compare_HAI.corr(method= "pearson", numeric_only=True)

In [ ]:
#Generate larger plot colored by day with both groups shown, but not indicated
alt.Chart(compare_HAI.loc[compare_HAI['virus'] =='A/Hawaii/70/2019']).mark_point(filled=True, size=75).encode(
    x=alt.X('NGS_titer', scale=alt.Scale(type="log"),axis=alt.Axis(title="Sequencing-based NT50")),
    y=alt.Y('HAI_titer', scale=alt.Scale(type="log"),axis=alt.Axis(title="HAI Titer")),
    color =alt.Color('timepoint:N', sort=['0','30'], title="days post vaccination",legend=alt.Legend(orient='right',labelLimit= 0)).scale(range = ['rebeccapurple', 'firebrick']),
).properties(width = 250, height = 250).configure_axis(grid=False, domain=False,labelFontSize=14,titleFontSize=20).configure_legend(titleAlign='left',labelLimit= 0,
titleFontSize=15,
labelFontSize=16
)

In [ ]:
#Designate order for conditions to be plotted
condition_order = ['Day 0','Day 30','Day 182']

Plot all the titers:

In [ ]:
#Here is a function that generates the interactive plot of titers by individual
def run_titerchart(dataframe, ncols=10):

    virus_selection = alt.selection_point(fields=["virus"], on="mouseover", empty=False)

    serum_selection = alt.selection_point(
        fields=["individual"],
        bind="legend",
        toggle="true",
    )

    sera = dataframe["individual"].unique().tolist()

    titers_chart = (
        alt.Chart(dataframe)
        .add_params(virus_selection, serum_selection)
        .transform_filter(serum_selection)
        .encode(
            alt.X(
                "titer",
                title="neutralization titer",
                scale=alt.Scale(nice=False, padding=4, type="log"),
                axis=alt.Axis(labelOverlap=True),
            ),
            alt.Y("virus", sort=viruses),
            alt.Color("condition", sort=condition_order, title="Timepoint",scale=alt.Scale(range=['rebeccapurple', 'firebrick', 'mediumseagreen'])),
            alt.Facet(
                "participant:N",
                header=alt.Header(
                    title=None, labelFontSize=11, labelFontStyle="bold", labelPadding=0
                ),
                spacing=3,
                columns=ncols,
            ),
            alt.StrokeWidth(
                "individual:N",
                scale=alt.Scale(domain=sera, range=[1] * len(sera)),
                legend=alt.Legend(
                    orient="bottom",
                    columns=ncols,
                    symbolLimit=0,
                    symbolFillColor="black",
                    title="participant (click to select)",
                ),
            ),
            tooltip=[
                alt.Tooltip(c, format=".3g") if titers[c].dtype == float else c
                for c in titers.columns
            ],
        )
        .mark_line(point=True)
        .configure_axis(grid=False)
        .configure_point(size=45)
        .properties(
            height=alt.Step(9),
            width=110,
            title=alt.TitleParams(
                "Interactive chart of serum neutralization titers",
                subtitle="Mouseover points for details, click serum legend at bottom to select sera to show",
                fontSize=15,
                dx=100,
                dy=-5,
            ),
            autosize=alt.AutoSizeParams(resize=True),
        )
    )

    return titers_chart

In [ ]:
def run_titerchart_rot(dataframe, ncols=10):

    virus_selection = alt.selection_point(fields=["virus"], on="mouseover", empty=False)

    serum_selection = alt.selection_point(
        fields=["individual"],
        bind="legend",
        toggle="true",
    )

    sera = dataframe["individual"].unique().tolist()

    titers_chart = (
        alt.Chart(dataframe)
        .add_params(virus_selection, serum_selection)
        .transform_filter(serum_selection)
        .encode(
            alt.Y(
                "titer",
                title="neutralization titer",
                scale=alt.Scale(nice=False, padding=4, type="log"),
                axis=alt.Axis(labelOverlap=True),
            ),
            alt.X("virus", sort=viruses),
            alt.Color("condition", sort=condition_order, title="Timepoint",scale=alt.Scale(range=['rebeccapurple', 'firebrick', 'mediumseagreen'])),
            alt.Facet(
                "participant:N",
                header=alt.Header(
                    title=None, labelFontSize=11, labelFontStyle="bold", labelPadding=0
                ),
                spacing=3,
                columns=ncols,
            ),
            alt.StrokeWidth(
                "individual:N",
                scale=alt.Scale(domain=sera, range=[1] * len(sera)),
                legend=alt.Legend(
                    orient="bottom",
                    columns=ncols,
                    symbolLimit=0,
                    symbolFillColor="black",
                    title="participant (click to select)",
                ),
            ),
            tooltip=[
                alt.Tooltip(c, format=".3g") if titers[c].dtype == float else c
                for c in titers.columns
            ],
        )
        .mark_line(point=True)
        .configure_axis(grid=False)
        .configure_point(size=45)
        .properties(
            width=alt.Step(11),
            height=80,
            title=alt.TitleParams(
                "Interactive chart of serum neutralization titers",
                subtitle="Mouseover points for details, click serum legend at bottom to select sera to show",
                fontSize=15,
                dx=100,
                dy=-5,
            ),
            autosize=alt.AutoSizeParams(resize=True),
        )
    )

    return titers_chart

In [ ]:
sera_withgap = ['D10175','D10193','D10244']
chart = run_titerchart_rot(titers_withgroups.loc[titers_withgroups["individual"].isin(sera_withgap)], 1)
selected_sera2 = ['D10066','D10011','D10366']

#selected_sera = ['D10396','D10191','D10256','D10041','D10042','D10159']
for df, chart_html, ncols in [
    (titers_withgroups.loc[titers_withgroups["individual"].isin(selected_sera2)], titers_chart_serawithgap_rotated_html, 1),
    (titers_withgroups.loc[titers_withgroups["individual"].isin(sera_withgap)], titers_chart_selectedserum_rotated_html, 1),
]:
    print("\n\n*********************************")
    chart = run_titerchart_rot(df, ncols)
    display(chart)
    print(f"Saving to {chart_html}")
    chart.save(chart_html)

In [ ]:
#Assign location for plot to be saved to and run chart
selected_sera = ['D10181','D10378','D10366']
selected_sera2 = ['D10066','D10011','D10366']
#selected_sera = ['D10396','D10191','D10256','D10041','D10042','D10159']
for df, chart_html, ncols in [
    (titers_withgroups, titers_chart_html, 10),
    (titers_withgroups.query("group == '1xVax'"), titers_chart_1xVax_html, 5),
    (titers_withgroups.query("group == '2xVax'"), titers_chart_2xVax_html, 5),
    (titers_withgroups.loc[titers_withgroups["individual"].isin(selected_sera)], titers_chart_selectedpeople_html, 3),
]:

    print("\n\n*********************************")
    chart = run_titerchart(df, ncols)
    display(chart)
    print(f"Saving to {chart_html}")
    chart.save(chart_html)

In [ ]:
#Pivot table to calculate fold-change make dataframes both with and without individuals who do not have a day 182 sample
NT50s_by_strain = titers_withgroups.pivot_table('titer', ['individual','virus','group'],'condition').reset_index()

NT50s_by_strain['foldchange_d30'] = NT50s_by_strain['Day 30']/NT50s_by_strain['Day 0']
NT50s_by_strain['foldchange_d182'] = NT50s_by_strain['Day 182']/NT50s_by_strain['Day 0']
NT50s_by_strain_w182 = NT50s_by_strain.dropna()
NT50s_by_strain_withmedian_w182 = NT50s_by_strain_w182.drop(columns='individual').groupby(['virus','group']).median().reset_index()

In [ ]:
#Make a dataframe that contains only individuals who have a day 182 sample
no182 = NT50s_by_strain.loc[NT50s_by_strain['Day 182'].isna()]['individual'].unique().tolist()
titers_withgroups_w182 = titers_withgroups.loc[~titers_withgroups['individual'].isin(no182)]

In [ ]:
#calculate the median for each virus group for all samples and for just those that have a day 182 sample
titers_withgroup_calculatemedian = titers_withgroups.drop(columns=['serum','individual','titer_sem','n_replicates','titer_bound','titer_as'])
titers_withgroup_calculatemedian = titers_withgroup_calculatemedian.groupby(['virus','group','condition']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'})
titers_withgroupdayandmedian = titers_withgroups.merge(titers_withgroup_calculatemedian, on=['virus','group','condition'])
titers_withgroupdayandmedian_allday30 = titers_withgroupdayandmedian.loc[~titers_withgroupdayandmedian['condition'].str.contains('182')]

titers_withgroup_calculatemedian_w182 = titers_withgroups_w182.drop(columns=['serum','individual','titer_sem','n_replicates','titer_bound','titer_as'])
titers_withgroup_calculatemedian_w182 = titers_withgroup_calculatemedian_w182.groupby(['virus','group','condition']).median(numeric_only=True).reset_index().rename(columns={'titer':'median_titer'})
titers_withgroup_calculatemedian_w182 = titers_withgroups_w182.merge(titers_withgroup_calculatemedian_w182, on=['virus','group','condition'])

In [ ]:
#Run chart for calculating fold change
source = NT50s_by_strain_w182
median_source = NT50s_by_strain_withmedian_w182
range_ = ['#7b4173', '#e45756']
range_ = ['steelblue','goldenrod']

domain_ = ['1xVax','2xVax']
plot_range = [0.5,120]

day_to_plot = "Day 30"

NT50s_forselections_chart = alt.Chart(source).mark_errorband(opacity=0.2,extent="iqr",).encode(
    y=alt.X('foldchange_d30',scale=alt.Scale(domain=plot_range, type="log",nice=False),axis=alt.Axis(grid=False,titleFontSize=12, labelFontSize=12)),
    x=alt.Y("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    color=alt.Color("group", title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart(median_source).mark_line(point=True,strokeWidth=1).encode(
    y=alt.X('foldchange_d30',scale=alt.Scale(domain=plot_range,type="log", nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart(median_source).mark_point(filled=True).encode(
    y=alt.X("foldchange_d30",scale=alt.Scale(domain=plot_range,type="log", nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))

chart =meanline_chart_point + NT50s_forselections_chart + meanline_chart
chart.layer[0].encoding.x.title = ' titer'
chart.layer[0].title = 'Fold-Change in NT50 with Vaccination'
chart.properties(
    width=80,
    height = alt.Step(11))
chart.save(titers_chart_foldchange_html)
chart

In [ ]:
#Run chart for calculating fold change at day 182
NT50s_forselections_chart = alt.Chart(source).mark_errorband(opacity=0.2,extent="iqr",).encode(
    y=alt.X('foldchange_d182',scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=12, labelFontSize=12)),
    x=alt.Y("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    color=alt.Color("group", title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart(median_source).mark_line(point=True,strokeWidth=1).encode(
    y=alt.X('foldchange_d182',scale=alt.Scale(type='log',domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart(median_source).mark_point(filled=True).encode(
    y=alt.X("foldchange_d182",scale=alt.Scale(type='log',domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))

chart =meanline_chart_point + NT50s_forselections_chart + meanline_chart
chart.layer[0].encoding.x.title = ' titer'
chart.layer[0].title = 'Fold-Change in NT50 with Vaccination'
chart.properties(
    width=80,
    height = alt.Step(11))
#chart.save(titers_chart_foldchange_html)
chart

In [ ]:
domain_ = ['1xVax','2xVax']
plot_range = [60,30000]
NT50s_forselections_chart = alt.Chart().mark_errorband(opacity=0.2,extent="iqr",).encode(
    alt.Y("titer",scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=14, labelFontSize=14, title="titer")),
    alt.X("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    alt.Color("group", title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.Y("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.X("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart().mark_point(filled=True).encode(
    y=alt.Y("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.X("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))


day_order = ['Day 0','Day 30','Day 182']
median_chart = alt.layer(meanline_chart_point,NT50s_forselections_chart, meanline_chart, data = titers_withgroupdayandmedian_allday30).facet('condition:N', columns=1)
median_chart.save(titers_chart_median_all_html)
display(median_chart)

In [ ]:
domain_ = ['1xVax','2xVax']
plot_range = [60,30000]
NT50s_forselections_chart = alt.Chart().mark_errorband(opacity=0.2,extent="iqr",).encode(
    alt.Y("titer",scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=14, labelFontSize=14, title="titer")),
    alt.X("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    alt.Color("group", title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.Y("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.X("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart().mark_point(filled=True).encode(
    y=alt.Y("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.X("virus", sort=viruses,
            title="virus"),
    color=alt.Color("group",title="group",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))


day_order = ['Day 0','Day 30','Day 182']
median_chart = alt.layer(meanline_chart_point,NT50s_forselections_chart, meanline_chart, data = titers_withgroup_calculatemedian_w182).facet('condition:N', columns=1)
median_chart.save(titers_chart_median_w182_html)
display(median_chart)

In [ ]:
#Make plot with all timepoints for all samples that have a Day 182 sample, facet by group
range_ = ['rebeccapurple', 'firebrick', 'mediumseagreen']
domain_ = ['Day 0','Day 30','Day 182']
plot_range = [60,30000]

NT50s_forselections_chart = alt.Chart().mark_errorband(opacity=0.2,extent="iqr",).encode(
    alt.Y("titer",scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=14, labelFontSize=14, title="titer")),
    alt.X("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    alt.Color("condition", title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.X("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("condition",title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart().mark_point(filled=True).encode(
    y=alt.X("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("condition",title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))



median_chart = alt.layer(meanline_chart_point,NT50s_forselections_chart, meanline_chart, data = titers_withgroup_calculatemedian_w182).facet(column=alt.Column('group', header=alt.Header(title=None, labelFontSize=12, labelFontStyle="bold", labelPadding=0)))
median_chart.save(titers_chart_splitbygroup_w182_html)
display(median_chart)

In [ ]:
#Make plot with all timepoints for all samples that have a Day 182 sample, facet by group

range_ = ['rebeccapurple', 'firebrick', 'mediumseagreen']
domain_ = ['Day 0','Day 30','Day 182']
plot_range = [60,30000]

NT50s_forselections_chart = alt.Chart().mark_errorband(opacity=0.2,extent="iqr",).encode(
    alt.Y("titer",scale=alt.Scale(type='log',domain=plot_range, nice=False),axis=alt.Axis(grid=False,titleFontSize=14, labelFontSize=14, title="titer")),
    alt.X("virus", sort=viruses, axis=alt.Axis(title=None,
                                                 labelFontSize=11,labelLimit=300)),
    alt.Color("condition", title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart = alt.Chart().mark_line(point=False,strokeWidth=1).encode(
    y=alt.X("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("condition",title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
).properties(
    height=80,
    width = alt.Step(11))
meanline_chart_point = alt.Chart().mark_point(filled=True).encode(
    y=alt.X("median_titer",scale=alt.Scale(domain=plot_range, nice=False)),
    x=alt.Y("virus", sort=viruses,
            title="virus"),
    color=alt.Color("condition",title="day",scale=alt.
                    Scale(domain=domain_, range=range_)),
    size=alt.value(45),
    opacity=alt.value(1),
).properties(
    height=80,
    width = alt.Step(11))



median_chart = alt.layer(meanline_chart_point,NT50s_forselections_chart, meanline_chart, data = titers_withgroupdayandmedian_allday30).facet(column=alt.Column('group', header=alt.Header(title=None, labelFontSize=12, labelFontStyle="bold", labelPadding=0)))
median_chart.save(titers_chart_splitbygroup_all_html)
display(median_chart)

In [ ]:
stats.ttest_ind(titers_withgroups_w182.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_withgroups_w182.query("group == '2xVax'").query("condition == 'Day 0'")['titer'], equal_var = False)

In [ ]:
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[titers_withgroups['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_withgroups_w182.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_withgroups_w182.query("group == '2xVax'").query("condition == 'Day 0'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest[1])

In [ ]:
#Just look at individuals with a day 182 sample
print("Statistically significant Day 0 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.ttest_ind(titers_totest.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 0'")['titer'], equal_var = False)
    if ttest[1] < 0.05:
        print(virus, ttest[1])
print("\n")
print("Statistically significant Day 30 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.ttest_ind(titers_totest.query("group == '1xVax'").query("condition == 'Day 30'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 30'")['titer'], equal_var = False)
    if ttest[1] < 0.05:
        print(virus, ttest[1])
print("\n")
print("Statistically significant Day 182 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.ttest_ind(titers_totest.query("group == '1xVax'").query("condition == 'Day 182'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 182'")['titer'], equal_var = False)
    if ttest[1] < 0.05:
        print(virus, ttest[1])

In [ ]:
print("Statistically significant Day 0 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.median_test(titers_totest.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 0'")['titer'],ties="ignore")
    if ttest[1] < 0.05:
        print(virus, ttest[1])
print("\n")
print("Statistically significant Day 30 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.median_test(titers_totest.query("group == '1xVax'").query("condition == 'Day 30'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 30'")['titer'],ties="ignore")
    if ttest[1] < 0.05:
        print(virus, ttest[1])
print("\n")
print("Statistically significant Day 182 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[~titers_withgroups_w182['individual'].isin(no182)].loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.median_test(titers_totest.query("group == '1xVax'").query("condition == 'Day 182'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 182'")['titer'],ties="ignore")
    if ttest[1] < 0.05:
        print(virus, ttest[1])

In [ ]:
print("Statistically significant Day 0 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 0'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)
print("\n")
print("Statistically significant Day 30 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 30'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 30'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)
print("\n")
print("Statistically significant Day 182 samples:")
for virus in viruses:
    titers_totest = titers_withgroups_w182.loc[titers_withgroups_w182['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 182'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 182'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)

In [ ]:
print("Statistically significant Day 0 samples:")
for virus in viruses:
    titers_totest = titers_withgroups.loc[titers_withgroups['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 0'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 0'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)
print("\n")
print("Statistically significant Day 30 samples:")
for virus in viruses:
    titers_totest = titers_withgroups.loc[titers_withgroups['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 30'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 30'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)
print("\n")
print("Statistically significant Day 182 samples:")
for virus in viruses:
    titers_totest = titers_withgroups.loc[titers_withgroups['virus'] == virus]
    ttest = stats.mannwhitneyu(titers_totest.query("group == '1xVax'").query("condition == 'Day 182'")['titer'], titers_totest.query("group == '2xVax'").query("condition == 'Day 182'")['titer'])
    if ttest[1] < 0.05:
        print(virus, ttest)